    1- Key 64 or 56 becomes KeyL and KeyR 28 each
    2- circular shift depending on round
    3- take circulated for round, take uncirculated for next round




In [46]:
import binascii
import numpy as np
import keyTables, Sbox, textTables, additionalFunction, functions

roundNumber = 1
C = [] #KeyLeft
D = [] #KeyRight
L = [] #PlainLeft
R = [] #PlainRight
roundKey =[]


In [4]:
#a
initialPermutation = [58, 50, 42, 34, 26, 18, 10, 2,
                      60, 52, 44, 36, 28, 20, 12, 4,
                      62, 54, 46, 38, 30, 22, 14, 6,
                      64, 56, 48, 40, 32, 24, 16, 8,
                      57, 49, 41, 33, 25, 17, 9, 1,
                      59, 51, 43, 35, 27, 19, 11, 3,
                      61, 53, 45, 37, 29, 21, 13, 5,
                      63, 55, 47, 39, 31, 23, 15, 7]
#b
inverseInitialPermutation = [40, 8, 48, 16, 56, 24, 64, 32,
                            39, 7, 47, 15, 55, 23, 63, 31,
                            38, 6, 46, 14, 54, 22, 62, 30,
                            37, 5, 45, 13, 53, 21, 61, 29,
                            36, 4, 44, 12, 52, 20, 60, 28,
                            35, 3, 43, 11, 51, 19, 59, 27,
                            34, 2, 42, 10, 50, 18, 58, 26,
                            33, 1, 41, 9, 49, 17, 57, 25]
#c
expansionPermutation     = [32, 1, 2, 3, 4, 5,
                            4, 5, 6, 7, 8, 9,
                            8, 9, 10, 11, 12, 13,
                            12, 13, 14, 15, 16, 17,
                            16, 17, 18, 19, 20, 21,
                            20, 21, 22, 23, 24, 25,
                            24, 25, 26, 27, 28, 29,
                            28, 29, 30, 31, 32, 1]
#d
permutationFunction = [16, 7, 20, 21, 29, 12, 28, 17,
                        1, 15, 23, 26, 5, 18, 31, 10,
                        2, 8, 24, 14, 32, 27, 3, 9,
                        19, 13, 30, 6, 22, 11, 4, 25]

#Key
permutationChoice1 = [57, 49, 41, 33, 25, 17, 9,
                    1, 58, 50, 42, 34, 26, 18,
                    10, 2, 59, 51, 43, 35, 27,
                    19, 11, 3, 60, 52, 44, 36,
                    63, 55, 47, 39, 31, 23, 15,
                    7, 62, 54, 46, 38, 30, 22,
                    14, 6, 61, 53, 45, 37, 29,
                    21, 13, 5, 28, 20, 12, 4]
                    
permutationChoice2 = [14, 17, 11, 24, 1, 5, 3, 28,
                    15, 6, 21, 10, 23, 19, 12, 4,
                    26, 8, 16, 7, 27, 20, 13, 2,
                    41, 52, 31, 37, 47, 55, 30, 40,
                    51, 45, 33, 48, 44, 49, 39, 56,
                    34, 53, 46, 42, 50, 36, 29, 32]

RoundShift= [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

In [5]:
K= [0,0,0,0,0,0,0,1,  0,0,1,0,0,0,1,1,  0,1,0,0,0,1,0,1,  0,1,1,0,0,1,1,1,  1,0,0,0,1,0,0,1,  1,0,1,0,1,0,1,1,  1,1,0,0,1,1,0,1,  1,1,1,0,1,1,1,1]
plainText = [0,0,0,0,0,0,0,1,  0,0,1,0,0,0,1,1,  0,1,0,0,0,1,0,1,  0,1,1,0,0,1,1,1,  1,0,0,0,1,0,0,1,  1,0,1,0,1,0,1,1,  1,1,0,0,1,1,0,1,  1,1,1,0,1,1,1,1]
print(K)
print(plainText)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]


In [43]:
def PC(Key, pcNumber):
    permutationChoice = []
    if(pcNumber == 1):
        permutationChoice = permutationChoice1
    else:
        permutationChoice = permutationChoice2


    arr = []
    for i in range(len(permutationChoice)):
       arr.append(Key[permutationChoice[i] -1])
    # for i in range(0, len(arr),8):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr

def IP(Plaintext):
    
    arr = []
    for i in range(len(initialPermutation)):
       arr.append(Plaintext[initialPermutation[i] -1])
    for i in range(0, len(arr),8):
        print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr

In [39]:
Key = PC(K,1)
C = Key[:28]
D = Key[28:]
# print(C)
# print(D)

In [35]:
def roundKeyFunc():
    roundKey = []
    for i in range(RoundShift[roundNumber  -1]):
        C.append(C.pop(0)) #Circular Shift Left of Left Key
        D.append(D.pop(0)) #Circular Shift Left of Right Key
    for i in range(len(C)):
        roundKey.append(C[i])
    for i in range(len(D)):
        roundKey.append(D[i])

    roundKey = PC(roundKey,2)
    return roundKey  
    
    

In [48]:
Initial = IP(plainText)
L=Initial[:32]
R=Initial[32:]
print(L)
print(R)

1 1 0 0 1 1 0 0
0 0 0 0 0 0 0 0
1 1 0 0 1 1 0 0
1 1 1 1 1 1 1 1
1 1 1 1 0 0 0 0
1 0 1 0 1 0 1 0
1 1 1 1 0 0 0 0
1 0 1 0 1 0 1 0
[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [40]:
roundKey = roundKeyFunc() #gets key for round

In [42]:
print(roundKey)

[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1]


In [112]:
s = 'Hello, World!'
b = string2bits(s)
s2 = bits2string(b)

In [113]:
a = listToString(b)
x = a.replace(" ", "")
len(x)


104

In [114]:


print('String:')
print(s)

print('\nList of Bits:')
for x in b:
    print (x)

print ('\nString:')
print (s2)

String:
Hello, World!

List of Bits:
01001000
01100101
01101100
01101100
01101111
00101100
00100000
01010111
01101111
01110010
01101100
01100100
00100001

String:
Hello, World!
